In [22]:
import torch
import torchvision

import matplotlib.pyplot as plt
import numpy as np
import time

from nni.retiarii import model_wrapper
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import random

import nni
import nni.retiarii.nn.pytorch as nn
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator import FunctionalEvaluator
import nni.retiarii.strategy as strategy



# Device configuration

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()

if num_gpus >= 2:
    # Select the second GPU (index 1)
    device = torch.device("cuda:1")
else:
    # If there are not enough GPUs, use the first one (index 0) or CPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [23]:
# Let's set parameters here
datapath = "whole_data.npy"
labelpath = "whole_label.npy"
NUM_FEATURES = 7
batch_size = 16
train_epochs = 10

In [29]:
np.load(datapath).shape

(12068, 100, 100, 7)

In [24]:
import nni.retiarii.nn.pytorch as nn
from nni.retiarii import model_wrapper

@model_wrapper
class ResNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.model_name = 'resnet'
        self.num_classes = num_classes
        
        num_channel_1 = nn.ValueChoice([32, 48, 64], label="num_channel")
        
        kernel_sizes = nn.ValueChoice([3, 5], label="kernel_size")
        strides = nn.ValueChoice([1, 2], label="stride")
        paddings = nn.ValueChoice([1, 2, 3], label="padding")
        self.conv1 = nn.Conv2d(NUM_FEATURES, num_channel_1, kernel_size=kernel_sizes, stride=strides, padding=paddings, bias=False)
        self.bn1 = nn.BatchNorm2d(num_channel_1)
        self.relu = nn.ReLU(inplace=True)
        
        kernel_size_pool = nn.ValueChoice([2, 3], label="kernel_size_pool")
        stride_size_pool = nn.ValueChoice([1, 2], label="stride_size_pool")
        self.poolchoice = nn.LayerChoice([nn.MaxPool2d(kernel_size=kernel_size_pool, stride=stride_size_pool, padding=1),
                                         nn.Identity()], label="poolchoice")
        
        num_channel_2 = num_channel_1*2
        num_channel_3 = num_channel_2*2
        num_channel_4 = num_channel_3*2

        ### layer1
        self.block1 = self.make_block(num_channel_1, num_channel_1, 1)
        self.block2 = self.make_block(num_channel_1, num_channel_1, 1)
        ### layer2
        self.block3 = self.make_block(num_channel_1, num_channel_2, 2)
        self.block4 = self.make_block(num_channel_2, num_channel_2, 1)
        ### layer3
        self.block5 = self.make_block(num_channel_2, num_channel_3, 2)
        self.block6 = self.make_block(num_channel_3, num_channel_3, 1)
        ### layer4
        self.block7 = self.make_block(num_channel_3, num_channel_4, 2)
        self.block8 = self.make_block(num_channel_4, num_channel_4, 1)
        
        ### downsample
        self.ds2 = nn.Sequential(nn.Conv2d(num_channel_1, num_channel_2, kernel_size=1, stride=2, bias=False),
                                 nn.BatchNorm2d(num_channel_2))
        self.ds3 = nn.Sequential(nn.Conv2d(num_channel_2, num_channel_3, kernel_size=1, stride=2, bias=False),
                                 nn.BatchNorm2d(num_channel_3))
        self.ds4 = nn.Sequential(nn.Conv2d(num_channel_3, num_channel_4, kernel_size=1, stride=2, bias=False),
                                 nn.BatchNorm2d(num_channel_4))
        
        self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(num_channel_4, num_classes)

    def make_block(self, inplanes, planes, stride=1):
        conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        bn1 = nn.BatchNorm2d(planes)
        relu = nn.ReLU(inplace=True)
        conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        bn2 = nn.BatchNorm2d(planes)
        basicblock = []
        basicblock += [conv1, bn1, relu, conv2, bn2]
        return nn.Sequential(*basicblock)
           
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.poolchoice(x)
        x1 = self.block1(x) + x
        x1 = self.block2(x1) + x1
        x2 = self.ds2(x1) + self.block3(x1)
        x2 = self.block4(x2) + x2
        x3 = self.ds3(x2) + self.block5(x2)
        x3 = self.block6(x3) + x3
        x4 = self.ds4(x3) + self.block7(x3)
        x4 = self.block8(x4) + x4
        x = self.adaptivepool(x4)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
        
        
        

In [26]:

def train_epoch(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    n_samples = 0
    n_correct = 0
    
    for i, (images, labels) in enumerate(train_loader): 
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Accuracy
        _, predicted = outputs.max(axis=1)
        n_samples += labels.size(0) 
        n_correct += (predicted == labels).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
def test_epoch(model, device, test_loader):
    model.eval()  
    n_correct = 0
    n_samples = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'======== Accuracy of the CNN: {acc} %')
    return acc



def evaluate_model(model_cls):
    model = model_cls()

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    data = np.load(datapath)  # (num, 100, 100, 5)
    labels = np.load(labelpath)  # (num,)
    data = np.transpose(data, (0, 3, 1, 2))
    
    # Convert data and labels to torch tensors
    data = torch.tensor(data, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.long)
                   
    # Define the number of folds for stratified k-fold
    k_folds = 5
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    accuracy = 0
                   
    for fold, (train_idx, val_idx) in enumerate(skf.split(data, labels)):
        # Split the dataset into training and validation sets
        train_data = TensorDataset(data[train_idx], labels[train_idx])
        val_data = TensorDataset(data[val_idx], labels[val_idx])

        # Create DataLoader for training and validation sets
        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False)
                   
        for epoch in range(train_epochs):
            # train the model for one epoch
            train_epoch(model, device, train_loader, criterion, optimizer, epoch)
            # test the model for one epoch
        accuracy += test_epoch(model, device, test_loader)
        # call report intermediate result. Result can be float or dict
        nni.report_intermediate_result(accuracy/(fold+1))
        
    cv_accuracy = accuracy/k_folds
    # report final test result
    nni.report_final_result(cv_accuracy)
    

In [ ]:

# Model HERE
model_space = ResNet()
evaluator = FunctionalEvaluator(evaluate_model)
search_strategy = strategy.Random(dedup=True)


exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'resnet18_drainage_crx'

exp_config.max_trial_number = 288
exp_config.trial_concurrency = 2  # will run two trials concurrently
exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True


# Start webUI
port = random.randint(1100,25000)
print ("running on port")
print(port)
exp.run(exp_config, port)


In [ ]:
for model_dict in exp.export_top_models(formatter='dict'):
    print(model_dict)